In [15]:
import pandas as pd
import numpy as np
import datetime as dt
from matplotlib import pyplot as plt
import seaborn as sns
from scipy.stats import expon
from sklearn.model_selection import GridSearchCV, LeaveOneOut, KFold

In [21]:
pedidos = pd.read_csv('./Dados_Ini/pedidos.csv', parse_dates=['order_creation_timestamp_local', 'col_date_local', 'cfm_date_local', 'dcl_date_local', 'ncl_date_local', 'con_date_local', 'can_date_local'])

In [22]:
pedidos['con_date_local'] = pedidos['con_date_local'].combine_first(pedidos['can_date_local'])

In [23]:
pedidos = pedidos[(pedidos['dish_type'] != 'Mercado') & np.isnan(pedidos['can_date_local']) & ~np.isnan(pedidos['dcl_date_local']) & (pedidos['dcl_date_local'] >= '2023-01-09') & (pedidos['dcl_date_local'] < '2023-01-12')]

In [24]:
pedidos = pedidos[['order_id', 'frn_id', 'delivery_by', 'dish_type', 'cfm_date_local', 'dcl_date_local', 'con_date_local', 'cli_lat', 'cli_lon', 'rst_lat', 'rst_lon']] \
    .rename(columns={'order_id': 'Ident. Pedido', 'frn_id': 'Estabelecimento', 'delivery_by': 'Tipo Entrega', 'dish_type': 'Tipo Estab.', 'cfm_date_local': 'Início Preparo', 'dcl_date_local': 'Início Entrega', 'con_date_local': 'Término Entrega', 'cli_lat': 'Lat Cliente', 'cli_lon': 'Lon Cliente', 'rst_lat': 'Lat Estab.', 'rst_lon': 'Lon Estab.'})

In [25]:
pedidos.to_csv('./Dados/Amostra_Pedidos.csv', index=False)

In [ ]:
pedidos['hr_pedido'] = pedidos.dcl_date_local.dt.floor('30min').dt.strftime('%H:%M')

In [ ]:
def gerar_amostra(df, grp, min_repr, n, semente):
    pop = df[grp].value_counts()
    np.random.seed(semente)
    pop_valida = np.random.choice(pop[pop.values >= min_repr].index, size=n, replace=False)
    amostra = df[df[grp].isin(pop_valida)]
    
    return amostra

In [ ]:
agrupador, pedidos_minimos, tam_amostra, semente = 'frn_id', 100, 3, 1234

In [ ]:
amostra_rests = gerar_amostra(pedidos, agrupador, pedidos_minimos, tam_amostra, semente)

In [ ]:
def plotar_eventos_ts(df_a, grp_a, ts, evt, cols, n):
    grupos = df_a.groupby(grp_a)
    fig, axs = plt.subplots(ncols=cols, nrows=int(n / cols), figsize=(20, (int(n / cols) + 1) * 3))

    for (grp, df), ax in zip(grupos, axs.ravel()):
        df.groupby(ts)[evt].count().plot(ax=ax, xlabel='', ylabel='', title=grp)

In [ ]:
plotar_eventos_ts(amostra_rests, 'frn_id', 'hr_pedido', 'order_id', 3, tam_amostra)

In [ ]:
KernelDensity?

In [ ]:
def estimar_h(df, grp, ts, evt):
    testes_h = 10 ** np.linspace(-1, 1, 100)
    grid = GridSearchCV(KernelDensity(kernel='gaussian'), {'bandwidth': testes_h}, cv=LeaveOneOut())
    grupos = df.groupby(grp)
    
    res = []
    for (grp, df) in grupos:
        Nt = df.groupby(ts).agg({evt: 'count'})
        grid.fit(Nt)
        
        res.append({'grupo': grp, 'h': grid})
    
    return res

In [ ]:
amostra_rests['t'] = pedidos.dcl_date_local.dt.floor('5min')
estimar_h(amostra_rests, 'frn_id', 't', 'order_id')